# Unsupervised learning exercise

This module is an exercise in unsupervised learning tools in sklearn.  The exercise is going to take county-level dairy sales data from the 2022 NASS Census dataset to group and label distinct dairy county labels.

In [5]:
# modules
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot as plt

import cattle_cycle_pkg.nass as nass
import cattle_cycle_pkg.env_vars as env_vars


In [6]:
# SQL engine and select statements

# create sql connection
user = env_vars.mktdb_user()
password = env_vars.mktdb_passwd()
server = env_vars.mktdb_server()

connection_string = f'postgresql://{user}:{password}@{server}/mktdb'
eng = create_engine(connection_string)
conn = eng.connect()

# Pull Dairy Data
dairy_sales_qry = """
select *
from nass_cen2022_fmt
where "GROUP_DESC" in ('DAIRY')
    and "AGG_LEVEL_DESC" in ('COUNTY')
    and "STATISTICCAT_DESC" in ('SALES')
    and "UNIT_DESC" in ('$')
    and "DOMAIN_DESC" in ('TOTAL')
    and "DOMAINCAT_DESC" in ('NOT SPECIFIED')
;
"""

dairy_co_df = pd.read_sql(dairy_sales_qry, con=conn)

print(dairy_co_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1770 entries, 0 to 1769
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   SOURCE_DESC            1770 non-null   object        
 1   SECTOR_DESC            1770 non-null   object        
 2   GROUP_DESC             1770 non-null   object        
 3   COMMODITY_DESC         1770 non-null   object        
 4   CLASS_DESC             1770 non-null   object        
 5   PRODN_PRACTICE_DESC    1770 non-null   object        
 6   UTIL_PRACTICE_DESC     1770 non-null   object        
 7   STATISTICCAT_DESC      1770 non-null   object        
 8   UNIT_DESC              1770 non-null   object        
 9   SHORT_DESC             1770 non-null   object        
 10  DOMAIN_DESC            1770 non-null   object        
 11  DOMAINCAT_DESC         1770 non-null   object        
 12  AGG_LEVEL_DESC         1770 non-null   object        
 13  STA

In [ ]:
# display(dairy_co_df.tail())
dairy_co_not_nan = dairy_co_df.dropna(subset=['VALUE'])
dairy_co_not_nan.groupby(['LOCATION_DESC', 'FULL_FIPS', 'STATE_FIPS', 'COUNTY_CODE'])['VALUE'].describe()

,,,,count,mean,std,min,25%,50%,75%,max
LOCATION_DESC,FULL_FIPS,STATE_FIPS,COUNTY_CODE,,,,,,,,
"ALABAMA, UPPER PLAINS & PIEDMONT, CHAMBERS",1 17.0,1,17.,1.0,750000.0,NaN,750000.0,750000.0,750000.0,750000.0,750000.0
"ARIZONA, NORTHERN, GILA",4 7.0,4,7.0,1.0,415000.0,NaN,415000.0,415000.0,415000.0,415000.0,415000.0
"ARIZONA, SOUTHERN, MARICOPA",4 13.0,4,13.,1.0,598803000.0,NaN,598803000.0,598803000.0,598803000.0,598803000.0,598803000.0
"ARIZONA, SOUTHERN, PINAL",4 21.0,4,21.,1.0,399708000.0,NaN,399708000.0,399708000.0,399708000.0,399708000.0,399708000.0
"ARKANSAS, CENTRAL, FAULKNER",5 45.0,5,45.,1.0,1334000.0,NaN,1334000.0,1334000.0,1334000.0,1334000.0,1334000.0
...,...,...,...,...,...,...,...,...,...,...,...
"WISCONSIN, WEST CENTRAL, PIERCE",5593.0,55,93.,1.0,64488000.0,NaN,64488000.0,64488000.0,64488000.0,64488000.0,64488000.0
"WISCONSIN, WEST CENTRAL, ST CROIX",55109.0,55,109,1.0,98919000.0,NaN,98919000.0,98919000.0,98919000.0,98919000.0,98919000.0
"WISCONSIN, WEST CENTRAL, TREMPEALEAU",55121.0,55,121,1.0,115456000.0,NaN,115456000.0,115456000.0,115456000.0,115456000.0,115456000.0
